In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tp2datostrain/test.csv
/kaggle/input/pingsound/PING - Sound effect.mp3


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from IPython.display import Audio
import matplotlib.pyplot as plt
from fastFM import als
from category_encoders import CatBoostEncoder

In [5]:
from IPython.display import Audio
# Cargo los archivos
sound_file = '/kaggle/input/pingsound/PING - Sound effect.mp3'
def ping():
    display(Audio(sound_file, autoplay=True))

In [14]:
es_para_submit = False
agregar_estimacion_preciom2 = True
if es_para_submit: 
    TAMANIO_TRAIN = 240000
    TAMANIO_TEST = 60000
else:
    TAMANIO_TRAIN = 190000
    TAMANIO_TEST = 50000

In [6]:
test = pd.read_csv("../input/tp2datostrain/test.csv")
display(test.shape)
train = pd.read_csv("../input/tp2datostrain/train.csv")
display(train.shape)

(60000, 22)

(240000, 23)

In [ ]:
ciudades = list(train["ciudad"].unique())
tipos_de_propiedad = list(train["tipodepropiedad"].unique())
provincia = list(train["provincia"].unique())
categorias  = [provincia, tipos_de_propiedad, ciudades]

In [ ]:
df = train

In [ ]:
features = [#cOLUMNAS
            'idzona',
            'lat', 'lng',
            "metrostotales", "metroscubiertos", 
            'garages', 'banos',
            'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
            'antiguedad',
            'habitaciones',
            'provincia', 'tipodepropiedad', 'ciudad',
           ]

In [ ]:
transformador_categorias = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                         OneHotEncoder(handle_unknown='ignore', categories = categorias))
transformador_numeros = make_pipeline(SimpleImputer())

In [ ]:
def obtenerFeaturesCategoricos(features):
    l = []
    for feature in features:
        if not np.issubdtype(df[feature].dtype, np.number):
            l.append(feature)
    return l

num_features = list(df[features].select_dtypes(include=[np.number]).columns)

cat_features = list(obtenerFeaturesCategoricos(features))
print ("Features numericos:", num_features, '\nFeatures categoricos: ', cat_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

In [ ]:
X = train[features].copy()
y = train["precio"].copy()
train_x, test_x, train_y, test_y = train_test_split(X, y,train_size=0.8, test_size=0.2)
from fastFM import als
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
x_procesada = preprocessor.fit_transform(train_x)
x_test_procesada = preprocessor.fit_transform(test_x)

fm.fit(x_procesada, train_y)
y_pred = fm.predict(x_test_procesada)

In [ ]:
ping()
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred, test_y)

In [ ]:
X = train[features].copy()
y = train["precio"].copy()
train_x, test_x, train_y, test_y = train_test_split(X, y,train_size=0.8, test_size=0.2)
from fastFM import sgd
fm2 = sgd.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, step_size=0.00000000001)
x_procesada = preprocessor.fit_transform(train_x)
x_test_procesada = preprocessor.fit_transform(test_x)

fm2.fit(x_procesada, train_y)
y_pred2 = fm2.predict(x_test_procesada)

In [ ]:
ping()
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred2, test_y)

PAra predecir los precios con los features normales no sirve de mucho FM

Pruebo un poco con el preprocesamiento y features mas copados

In [23]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [24]:
df.provincia.fillna('provinciaVacia', inplace = True)
aux2 = df[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
ciuds = aux2.T.to_dict('records').copy()
ciuds = ciuds[0]
df.ciudad.fillna(df.provincia.map(ciuds), inplace = True)
df.ciudad.fillna('ciudadVacia', inplace=True)

df.tipodepropiedad.fillna('tipoVacio', inplace=True)

aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

#Adicionales
aux = df.copy()
aux.descripcion.fillna('',inplace=True)
aux['jardin'] = (aux.descripcion.str.contains('jardin') | aux.descripcion.str.contains('jardín')).astype(int)
aux['vigilancia'] = (aux.descripcion.str.contains('vigilancia') | aux.descripcion.str.contains('seguridad')).astype(int) 
aux['buenaUbicacion'] = (aux.descripcion.str.contains('ubicacion') | aux.descripcion.str.contains('ubicación')).astype(int)
aux['lavadero'] = (aux.descripcion.str.contains('lavadero') | aux.descripcion.str.contains('lavado')).astype(int)
aux['jacuzzi'] = (aux.descripcion.str.contains('jacuzzi')).astype(int)
def buscar_barrioCerrado(string):
    return int(('barrio cerrado' in string) or ('urbanizacion cerrada' in string) or ('urbanización cerrada' in string)
    or ('barrio residencial' in string) or ('country' in string) or ('barrio privado' in string) or ('colonia privada' in string))
aux['barrio_cerrado'] = aux.descripcion.map(lambda x:buscar_barrioCerrado(str(x)))

valor_adicionales = {'gimnasio':4,'usosmultiples':2,'piscina':5,'escuelascercanas':3,'centroscomercialescercanos':3,
                     'buenaUbicacion':2,'vigilancia': 5,'jardin':4,'jacuzzi':3,'lavadero':0.5,'barrio_cerrado':10}
            
for columna in aux.columns:
    if columna in valor_adicionales:
        aux[columna] = aux[columna].map(lambda x: x*valor_adicionales[columna])

aux['adicionales'] = aux[valor_adicionales.keys()].sum(axis=1)
df['adicionales'] = aux['adicionales']


aux = df[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

aux = df[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

aux = df[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

aux = df[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)
df.garages.fillna(0, inplace=True)

df.drop(columns=['direccion'], inplace=True)


aux = df[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

aux = df[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)
df.banos.fillna(1, inplace=True)

aux = df.copy()
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']



df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

df['publicacionesporZonaYAnio'] = df.groupby(['idzona','anio'])['antiguedad'].transform('count')

aux = df[['mes', 'anio']].copy()
aux['cantidad'] =  1
aux['cantidad'] = aux.groupby(['mes', 'anio']).transform('sum')
df['publicacionesEnMes'] = aux['cantidad']

aux = df[['metrostotales', 'tipodepropiedad']].copy()
aux['metrosPromedioPorTipo'] = aux.groupby('tipodepropiedad').transform('mean')
df['metrosPromedioPorTipo'] = aux['metrosPromedioPorTipo']



aux = df[['metrostotales', 'ciudad']].copy()
aux['metrosPromedioPorCiudad'] = aux.groupby('ciudad').transform('mean')
df['metrosPromedioPorCiudad'] = aux['metrosPromedioPorCiudad']



In [25]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])



preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng','adicionales',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])



In [26]:
model = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)

In [27]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)


In [28]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [29]:
my_pipeline.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [30]:
preds = my_pipeline.predict(train_x)

In [31]:
from sklearn.metrics import mean_absolute_error
ping()
mean_absolute_error(preds, train_y)

919245.6012502837

Vemos que aún con los features utilizados en unas buenas estimaciones que tuvimos FM no da buen resultado.

OPeero podríamos usarlo para clasificar o predecir texto

### Trato de generar un feature que sea una aproximacion del precio en funcion del texto

In [62]:
aux = dfCheto[["titulo", "descripcion", "direccion", "precio"]].copy()
aux.fillna("",inplace = True)
aux["texto"] = aux["titulo"].astype(str) + " " + aux["descripcion"].astype(str) +" "+ aux["direccion"].astype(str)

In [63]:
import string
tabla = {}
for c in string.punctuation + "¿¡":
    tabla[ord(c)] = None
for c in string.ascii_uppercase:
    tabla[ord(c)] = c.lower()
def sacar_puntuacion_y_minusculizar(s):
    return s.translate(tabla)

aux["texto"] = aux["texto"].apply(sacar_puntuacion_y_minusculizar)

from nltk.corpus import stopwords
palabras_vacias = stopwords.words('spanish')
import unidecode
for i in range(len(palabras_vacias)):
    palabras_vacias[i] = unidecode.unidecode(palabras_vacias[i])

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words=palabras_vacias)

transformador_texto = Pipeline(steps=[('tf-idf', vectorizer)])
procesador_texto = ColumnTransformer(transformers=[
        ('text', transformador_texto, ['texto'])])

In [68]:
model = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
pipeline_texto = Pipeline(steps=[('preprocessor', procesador_texto),
                              ('model', model)
                             ])


In [95]:
entrenamiento = aux.head(TAMANIO_TRAIN).copy()
train_x = entrenamiento["texto"].copy()
train_y = entrenamiento.precio.copy()

texto_procesado = transformador_texto.fit_transform(train_x)

In [72]:
model.fit(texto_procesado, train_y)

FMRegression(init_stdev=0.1, l2_reg=0, l2_reg_V=0.5, l2_reg_w=0.1, n_iter=2000,
             random_state=123, rank=2)

In [75]:
preds = model.predict(texto_procesado)
ping()

In [77]:
from sklearn.metrics import mean_absolute_error
ping()
mean_absolute_error(preds, train_y)

300519.1238122829

In [96]:
prueba = aux.tail(60000).copy()
prueba.drop(columns=['precio'], inplace=True)
texto_procesado_test = transformador_texto.transform(prueba["texto"])
texto_procesado_test

<60000x197984 sparse matrix of type '<class 'numpy.float64'>'
	with 2371615 stored elements in Compressed Sparse Row format>

In [97]:
preds2 = model.predict(texto_procesado_test)

In [ ]:
ping()

In [99]:
res = pd.DataFrame(preds2, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

AttributeError: 'DataFrame' object has no attribute 'id'